In [1]:
from keras.layers import Embedding, Dense, Dropout, Input#, LSTM, Bidirectional
from keras.layers import MaxPooling1D, Conv1D, Flatten, LSTM
from keras.preprocessing import sequence, text
from keras.models import Model
from time import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split


import tensorflow as tf
print(tf.__version__)
np.random.seed(42)

In [3]:
data=pd.read_csv('../input/dialect-data/dialect_dataset with cleaning text.csv')

In [4]:
data=data.dropna()

In [5]:
from sklearn.model_selection import train_test_split,GridSearchCV
feature= data['text']
target=data['dialect']
x_train, x_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, stratify=target, random_state=42)

In [7]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
arabic= stopwords.words('arabic')


In [8]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten

In [9]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='char',stop_words=arabic , ngram_range=(3, 5), min_df=.000008, max_df=.998)
vectorizer.fit(x_train)
train= vectorizer.transform(x_train)
test = vectorizer.transform(x_test)

In [10]:
y_train

In [11]:
from sklearn.preprocessing import LabelBinarizer

y_train2 = LabelBinarizer().fit_transform(y_train)


In [42]:
y_train3= pd.get_dummies(y_train)
y_test3 = pd.get_dummies(y_test)
print(y_test3.shape)
print(y_train3.shape)

In [13]:
y_train2.shape

In [15]:
test.shape

In [16]:
shape=train.shape[1]

In [22]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x_train)
Xcnn_train = tokenizer.texts_to_sequences(x_train)
Xcnn_test = tokenizer.texts_to_sequences(x_test)
vocab_size = len(tokenizer.word_index) + 1  
print(x_train[1])
print(Xcnn_train[1]) 

In [23]:
from keras.preprocessing.sequence import pad_sequences
maxlen = 100
Xcnn_train = pad_sequences(Xcnn_train, padding='post', maxlen=maxlen)
Xcnn_test = pad_sequences(Xcnn_test, padding='post', maxlen=maxlen)
print(Xcnn_train[0, :]) 

In [112]:
from keras.models import Sequential
from keras import layers
embedding_dim = 200
textcnnmodel = Sequential()
textcnnmodel.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
textcnnmodel.add(layers.Conv1D(128, 5,activation='relu'))
textcnnmodel.add(layers.GlobalMaxPooling1D())
textcnnmodel.add(layers.Dense(10,activation='relu'))
textcnnmodel.add(layers.Dense(10, activation='relu'))
textcnnmodel.add(layers.Dense(18, activation='sigmoid'))

In [113]:
textcnnmodel.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

textcnnmodel.summary() 

In [114]:
textcnnmodel.fit(Xcnn_train, y_train2,
                     epochs=10,
                     validation_data=(Xcnn_test, y_test3),
                     batch_size=128)

In [117]:
loss, accuracy = textcnnmodel.evaluate(Xcnn_train, y_train3, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = textcnnmodel.evaluate(Xcnn_test, y_test3, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy)) 